## Librerías

In [1]:
%run "../recurrentes.ipynb"

In [2]:
%run "../funciones.ipynb"

In [3]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l

data = pd.read_parquet(dataset_lags_clase_ternaria_l)

## Valores null por variable

In [5]:
def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    breakpoints = sorted(set(
    [edge.left for edge in bin_edges if isinstance(edge, pd.Interval)] + 
    [edge.right for edge in bin_edges if isinstance(edge, pd.Interval)]))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null

In [6]:
# Ingresar mes train y mes score
train_data = data[data['foto_mes'] == 202108]
score_data = data[data['foto_mes'] < 202106]

train_null_percentage = train_data.isnull().mean() * 100
score_null_percentage = score_data.isnull().mean() * 100

comparison_df = pd.DataFrame({'Train Null Percentage': train_null_percentage, 'Score Null Percentage': score_null_percentage})
comparison_df['diff'] = (comparison_df['Score Null Percentage'] - comparison_df['Train Null Percentage']).abs()

comparison_df_sorted = comparison_df.sort_values('diff', ascending=False)

comparison_df_sorted

,Train Null Percentage,Score Null Percentage,diff
cmobile_app_trx,0.000000,17.943536,17.943536
tmobile_app,0.000000,17.943536,17.943536
Master_fultimo_cierre_lag_2,11.271624,18.845208,7.573584
Master_mpagominimo_lag_2,11.245633,18.787497,7.541865
Master_Fvencimiento_lag_2,11.245633,18.787497,7.541865
...,...,...,...
ccuenta_debitos_automaticos,0.000000,0.000000,0.000000
cpayroll2_trx,0.000000,0.000000,0.000000
mpayroll2,0.000000,0.000000,0.000000
mpayroll,0.000000,0.000000,0.000000


## Análisis drifting con PSI

In [7]:
psi_results = []
for column in train_data.columns:
  if column not in ['foto_mes', 'clase_ternaria']:
    train_variable = train_data[column]
    score_variable = score_data[column]
    psi_value = psi(train_variable, score_variable)
    psi_results.append({'feature': column, 'psi': psi_value})

psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)


In [8]:
f = psi_df['psi'] > 0.2
lista_variables_con_drift = psi_df.loc[f, 'feature'].to_list()

In [9]:
lista_variables_con_drift

['Master_fultimo_cierre_lag_1',
 'Visa_fultimo_cierre_lag_1',
 'Visa_Finiciomora_lag_2',
 'Master_Finiciomora',
 'mcomisiones_mantenimiento_lag_2',
 'Master_fultimo_cierre',
 'Visa_fultimo_cierre_lag_2',
 'Visa_Finiciomora_lag_1',
 'Master_fultimo_cierre_lag_2',
 'Visa_fultimo_cierre',
 'mcomisiones_mantenimiento_lag_1',
 'mpayroll_lag_2',
 'mcomisiones_mantenimiento',
 'Master_Fvencimiento_lag_2',
 'Master_Fvencimiento_lag_1',
 'mcomisiones_otras_lag_2',
 'Master_Fvencimiento',
 'mcomisiones_otras_lag_1',
 'ccomisiones_otras_lag_2',
 'mtransferencias_recibidas_lag_2',
 'mautoservicio_lag_1',
 'mcomisiones',
 'mcomisiones_otras',
 'mrentabilidad_annual_lag_2',
 'mautoservicio_lag_2',
 'ctransferencias_recibidas_lag_2',
 'mtransferencias_recibidas_lag_1',
 'mtarjeta_visa_consumo_lag_2',
 'mrentabilidad_lag_2',
 'mrentabilidad_annual_lag_1',
 'ccomisiones_otras_lag_1',
 'mtransferencias_emitidas_lag_2',
 'cpayroll_trx_lag_2',
 'chomebanking_transacciones_lag_2',
 'ctransferencias_emitida

In [10]:
variable_name = 'Master_Finiciomora'
expected = train_data[variable_name]
actual = score_data[variable_name]

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=10, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

print(f'Cortes en {variable_name}: {breakpoints}')
expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

print(f'Frecuencia Esperada: {expected_counts}')
print(f'Frecuencia Actual: {actual_counts}')


Cortes en Master_Finiciomora: [5.999, 15.0, 22.0, 29.0, 50.0, 57.0, 239.0]
Frecuencia Esperada: [ 27  59 313  46  94  66]
Frecuencia Actual: [2215 8197 2895 1832 1923 1116]


## Analisis drifting con modelo

In [11]:
# 2- Para el caso de que el dataset sea con feature engineering incluido
if 'T_visa_normal' in data.columns:
    data['T_visa_normal'] = data['T_visa_normal'].astype(bool)
    data['T_master_normal'] = data['T_master_normal'].astype(bool)


In [12]:
# 3- parametros para calcular ganancia del modelo
ganancia_acierto = 273000
costo_estimulo = 7000


data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001


In [13]:
# 4 - binarización de la clase ternaria

data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

In [14]:
# 5 Especificar mes de train y test

df_train = data[data['foto_mes']<=202106]
df_test = data[data['foto_mes']==202108]


In [15]:
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']

In [16]:
w_train = df_train.loc[X_train.index, 'clase_peso']


In [17]:
best_params = {'num_leaves': 40,
 'learning_rate': 0.022596031021514126,
 'min_data_in_leaf': 1692,
 'feature_fraction': 0.8060065372089812,
 'bagging_fraction': 0.8663914349386255}

In [18]:
train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)


params = {
'objective': 'binary',
'boosting_type': 'gbdt',
'first_metric_only': True,
'boost_from_average': True,
'feature_pre_filter': False,
'max_bin': 31,
'num_leaves': 0,
'learning_rate': 0,
'min_data_in_leaf': 0,
'feature_fraction': 0,
'bagging_fraction': 0,
'seed': semillas[0],
'verbose': 0
}
    #actualización con valores optuna
params.update(best_params)

if params['num_leaves'] == 0:
    print ('Error falta actualización de parámetros')


model = lgb.train(params,
                train_data,
                num_boost_round=299)



ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: T_Visa_normal: object, T_Master_normal: object

In [ ]:
model_df = model.trees_to_dataframe()

In [ ]:
model_df[model_df['split_feature'] == 'cpayroll_trx'].sort_values('threshold')

,tree_index,node_depth,node_index,left_child,right_child,parent_index,split_feature,split_gain,threshold,decision_type,missing_direction,missing_type,value,weight,count
12,0,6,0-S30,0-L23,0-L31,0-S22,cpayroll_trx,163.298004,1.000000e-35,<=,left,None,-1.799880,5183.05,41951
13017,164,3,164-S12,164-S13,164-L13,164-S1,cpayroll_trx,143.393997,1.000000e-35,<=,left,None,-0.000013,5085.28,208007
12958,164,3,164-S3,164-S10,164-L4,164-S2,cpayroll_trx,581.468018,1.000000e-35,<=,left,None,-0.006037,4965.61,518896
12776,161,3,161-S17,161-S20,161-L18,161-S1,cpayroll_trx,150.694000,1.000000e-35,<=,left,None,-0.000031,5100.99,208007
12721,161,3,161-S3,161-S12,161-L4,161-S2,cpayroll_trx,604.562988,1.000000e-35,<=,left,None,-0.006386,5073.26,518896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18002,227,7,227-S37,227-L29,227-L38,227-S28,cpayroll_trx,34.362701,1.500000e+00,<=,left,None,-0.003705,3715.52,551286
6976,88,6,88-S10,88-S11,88-S17,88-S7,cpayroll_trx,891.895996,1.500000e+00,<=,left,None,0.006705,60419.80,1096737
9120,115,7,115-S11,115-S12,115-S35,115-S9,cpayroll_trx,509.358002,1.500000e+00,<=,left,None,0.005579,39307.60,1072655
19260,243,8,243-S21,243-S36,243-L22,243-S18,cpayroll_trx,43.952099,2.500000e+00,<=,left,None,-0.002559,8438.13,1651612


## Analisis SHAPs

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

In [ ]:
shap_df = pd.DataFrame(shap_values, columns = Xtrain.columns)
shap_df

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,0.000108,0.0,-0.000064,0.0,0.000780,-0.000327,-0.000036,-0.006931,0.111702,-0.000217,...,0.0,0.000030,-0.000422,-0.004568,0.0,-0.002001,0.0,-0.000036,0.0,0.000400
1,0.000782,0.0,-0.000035,0.0,0.000828,-0.001488,0.000260,0.009638,-0.060537,-0.000213,...,0.0,0.000030,-0.000323,0.002208,0.0,-0.009880,0.0,-0.000246,0.0,0.002237
2,-0.000954,0.0,-0.000166,0.0,0.003445,-0.004733,0.000036,-0.019872,0.030292,-0.002095,...,0.0,-0.000116,-0.000138,-0.000053,0.0,-0.002781,0.0,0.001000,0.0,0.021907
3,0.000786,0.0,-0.000116,0.0,0.004378,-0.016219,0.000292,0.021660,0.053400,0.029521,...,0.0,0.000194,-0.000279,0.001874,0.0,0.003535,0.0,-0.003775,0.0,-0.032580
4,0.002799,0.0,-0.000027,0.0,0.001109,-0.003192,-0.001338,0.008862,-0.075301,0.000561,...,0.0,-0.000048,-0.000098,-0.000334,0.0,0.014763,0.0,0.012885,0.0,0.028679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164085,-0.000697,0.0,-0.000064,0.0,0.000781,-0.001061,0.000327,-0.006617,0.029540,-0.000164,...,0.0,-0.000029,0.000156,-0.000192,0.0,0.007741,0.0,-0.000065,0.0,-0.001508
164086,-0.000613,0.0,-0.000044,0.0,0.000793,0.004531,0.001169,-0.004326,-0.029545,-0.000161,...,0.0,0.000018,-0.000142,0.000178,0.0,0.004849,0.0,-0.000080,0.0,0.008610
164087,-0.001724,0.0,-0.000035,0.0,0.000882,-0.002153,0.000243,0.085012,0.190670,0.002264,...,0.0,0.000031,-0.001601,-0.000528,0.0,0.026746,0.0,-0.000117,0.0,0.024237
164088,-0.000756,0.0,-0.000035,0.0,0.000775,0.000666,0.000348,0.000745,-0.008159,-0.000151,...,0.0,0.000018,-0.000128,0.000065,0.0,0.012150,0.0,-0.000147,0.0,-0.001400


Calculamos la importancia de las variables usando SHAP


In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)
shap_df = pd.DataFrame(shap_values, columns = X_train.columns)

shap_importancia = pd.DataFrame(np.abs(shap_values).mean(0), columns=['SHAP Importance'])
shap_importancia['Feature'] = X_train.columns
shap_importancia.sort_values('SHAP Importance', ascending=False, inplace=True)
shap_importancia



,SHAP Importance,Feature
51,0.301123,cpayroll_trx
107,0.238453,ctrx_quarter
11,0.057666,mpasivos_margen
8,0.054274,mrentabilidad_annual
22,0.053195,mcuentas_saldo
...,...,...
87,0.000000,mcheques_depositados
86,0.000000,ccheques_depositados
83,0.000000,mtransferencias_emitidas
82,0.000000,ctransferencias_emitidas
